# Постановка задачи

Реализовать сервис multi-armed bandit для рекомендательного сервиса художников

# EDA

Check input data

In [1]:
!ls /srv/data/service_data

content_db.csv.gz  exhibitions_db.csv.gz  tags_db.csv.gz


In [47]:
import os

import pandas as pd

root_data_dir = '/srv/data/service_data'

content_db_df = pd.read_csv(os.path.join(root_data_dir, 'content_db.csv.gz'), compression='gzip')
content_db_df['art_tags'].fillna(value='', inplace=True)
print(content_db_df.shape[0])
content_db_df.head()

3540


,artist_name,art movement,nationality,field,artist_pic,wikipedia,artist_url,artist_name_check,artworks_url,artworks,artist_field,artist_movement,art_movement_raw_tags,art_tags
0,Hans von Aachen,"northern renaissance , mannerism (late renaiss...",German,painting,https://uploads8.wikiart.org/temp/e4d609cf-bbc...,en.wikipedia.org/wiki/Hans_von_Aachen,https://www.wikiart.org/en/hans-von-aachen,Hans von Aachen,https://www.wikiart.org/en/hans-von-aachen/all...,"[""https://uploads5.wikiart.org/images/hans-von...",painting,(Late Mannerism Northern Renaissance Renaissance),"['northern renaissance', 'mannerism (late rena...","northern renaissance,renaissance) (late manner..."
1,Vilmos Aba-Novak,expressionism,Hungarian,"fresco painting, fresco, graphics graphics pai...",https://uploads4.wikiart.org/images/vilmos-aba...,en.wikipedia.org/wiki/Vilmos_Aba-Novák,https://www.wikiart.org/en/vilmos-aba-novak,Vilmos Aba-Novak,https://www.wikiart.org/en/vilmos-aba-novak/al...,"[""https://uploads2.wikiart.org/images/vilmos-a...",fresco graphics painting,Expressionism,['expressionism'],expressionism
2,Reza Abbasi,"safavid period (before 1600), safavid and qaja...",Iranian,painting,https://uploads8.wikiart.org/temp/b08c3cef-4f3...,en.wikipedia.org/wiki/Reza_Abbasi,https://www.wikiart.org/en/reza-abbasi,Reza Abbasi,https://www.wikiart.org/en/reza-abbasi/all-wor...,"[""https://uploads2.wikiart.org/images/reza-abb...",painting,(after (before 1600) Period Periods Qajar Safa...,"['safavid period (before 1600)', 'safavid and ...","period (before 1600) safavid,qajar safavid (af..."
3,Giuseppe Abbati,", impressionism impressionism, realism realism",Italian,painting,https://uploads7.wikiart.org/00277/images/gius...,en.wikipedia.org/wiki/Giuseppe_Abbati,https://www.wikiart.org/en/giuseppe-abbati,Giuseppe Abbati,https://www.wikiart.org/en/giuseppe-abbati/all...,"[""https://uploads7.wikiart.org/00313/images/gi...",painting,Impressionism Realism,"['impressionism impressionism', 'realism reali...","impressionism,realism"
4,Vincenzo Abbati,romanticism,Italian,painting,https://uploads8.wikiart.org/00335/images/vinc...,en.wikipedia.org/wiki/Vincenzo_Abbati,https://www.wikiart.org/en/vincenzo-abbati,Vincenzo Abbati,https://www.wikiart.org/en/vincenzo-abbati/all...,"[""https://uploads4.wikiart.org/00330/images/vi...",painting,Romanticism,['romanticism'],romanticism


In [86]:
for _, row in content_db_df.sample(3).iterrows():
    print(row['artist_name'], row['artist_url'], 'https://'+row['wikipedia'])

Anne Brigman https://www.wikiart.org/en/anne-brigman https://en.wikipedia.org/wiki/Anne_Brigman
Hans Haacke https://www.wikiart.org/en/hans-haacke https://en.wikipedia.org/wiki/Hans_Haacke
Cagnaccio di San Pietro https://www.wikiart.org/en/cagnaccio-di-san-pietro https://en.wikipedia.org/wiki/Cagnaccio_di_San_Pietro


Колонка art_tags содержит тэги художника

Для удобства есть отдельный датафрейм с тэгами (`cnt` - counter, сколько присутствует в катклоге контента с таким тэгом)

In [37]:
tags_df = pd.read_csv(os.path.join(root_data_dir, 'tags_db.csv.gz'), compression='gzip')
print(tags_df.shape[0])

tags_df.head()

162


,tag,cnt
0,expressionism,558
1,realism,487
2,romanticism,343
3,contemporary,288
4,impressionism,287


Что в конце списка? Низкочастотные тэги

In [38]:
tags_df[-10:]

,tag,cnt
152,futuretech art,2
153,neo-conceptualism,2
154,brut) art (art naïve (primitivism) outsider,2
155,period (c.1370–1507) timurid,2
156,medievialism new,2
157,net.art,2
158,kingdoms dynasties (907–960) five ten and,2
159,art medieval,2
160,guro ero,2
161,joseon dynasty,2


# Business logic

Алгоритм $\varepsilon$-greedy рекомендаций multi-armed bandit

* Выбираем склонность алгоритма к explore $\varepsilon$, например $\varepsilon=0.2$
* С вероятностью $\varepsilon$ выбираем рандомный тэг, с вероятностью $1 - \varepsilon$ выбираем "лучший" тэг
* фильтруем элементы каталога с тэгом и получаем рекомендованных контент


In [53]:
random_tag = tags_df['tag'].sample(1).iloc[0]
print('Random tag: %s' % random_tag)  # sampling tag
cols = ['artist_name', 'art_tags', 'art movement', 'field']
# select content with sampled tag
(
    content_db_df[
        content_db_df['art_tags'].apply(lambda x: random_tag in x)
    ]
    [cols]
    .head()
)

Random tag: art street


,artist_name,art_tags,art movement,field
32,Jef Aerosol,art street,street art,"drawing painting, printmaking, design, drawing..."
178,Alaa Awad,"contemporary,contemporary,art street",", contemporary, street art street art contempo...","drawing painting, drawing, mosaic, fresco fres..."
217,Banksy,art street,street art,"drawing painting painting, printmaking, sculpt..."
245,Jean-Michel Basquiat,"neo-expressionism,neo-expressionism,art street...","neo-expressionism , street art neo-expressioni...",painting
715,Cope2,art street,street art,painting


Как получать "лучший" тэг? Для этого нужно собирать статистику взаимодействия пользователей с сервисом: лайки и дизлайки

Далее сортируем базу контента по убыванию количества лайков

Среди top-10 контента выбираем наиболее чатый тэг - это и будет самый лучший тэг



Реализуем алгорим

* имитируем лайки и дизлайки пользователя
* определяем лучший тэг
* сэмплируем контент

In [66]:
import numpy as np

history_length = 10

# модель реакций пользователя - лайки, дизлайки
LIKE = '🤩'
DISLIKE = '🥴'
action_type = [LIKE, DISLIKE]
like_probability = 0.7
action_probas = [like_probability, 1. - like_probability]

user_history = []
for _ in range(history_length):
    # выбираем контент чтобы показать пользователю - рандом
    random_tag = tags_df['tag'].sample(1).iloc[0]
    random_content = content_db_df[content_db_df['art_tags'].apply(lambda x: random_tag in x)].sample(1)
    # моделируем поведение пользователя
    user_action = np.random.choice(action_type, p=action_probas)
    user_action_json = {'action': user_action, 'content_tag': random_tag}
    print("Action added: %s" % user_action_json)
    user_history.append(user_action_json)
print("Num likes: %d" % len([i for i in user_history if i['action'] == LIKE]))

Action added: {'action': '🥴', 'content_tag': 'futuretech art'}
Action added: {'action': '🤩', 'content_tag': 'art neo-pop'}
Action added: {'action': '🤩', 'content_tag': 'transavantgarde'}
Action added: {'action': '🥴', 'content_tag': 'light and space'}
Action added: {'action': '🤩', 'content_tag': 'precisionism'}
Action added: {'action': '🥴', 'content_tag': 'neoclassicism'}
Action added: {'action': '🥴', 'content_tag': 'art conceptual'}
Action added: {'action': '🥴', 'content_tag': 'futuretech art'}
Action added: {'action': '🤩', 'content_tag': 'minimalism'}
Action added: {'action': '🥴', 'content_tag': 'new art media'}
Num likes: 4


Выбираем наиболее "подходящий" тэг и контент для этого тэга

In [73]:
import pandas as pd

def user_tags_ranking(user_actions, all_tags_df: pd.DataFrame):
    tags_df = all_tags_df.copy()
    if len(user_actions) > 0:
        user_negative_tags = pd.json_normalize([i for i in user_actions if i['action']==DISLIKE])
        if user_negative_tags.shape[0] > 0:
            user_negative_tags = user_negative_tags['content_tag'].value_counts().to_frame(name='cnt').reset_index()
            user_negative_tags.columns = ['content_tag', 'cnt']
        user_positive_tags = pd.json_normalize([i for i in user_actions if i['action']==LIKE])
        if user_positive_tags.shape[0] > 0:
            user_positive_tags = user_positive_tags['content_tag'].value_counts().to_frame(name='cnt').reset_index()
            user_positive_tags.columns = ['content_tag', 'cnt']
        if user_negative_tags.shape[0] > 0:  # drop disliked tags
            tags_df = (
                tags_df
                .merge(user_negative_tags, how='left', left_on='tag', right_on='content_tag',suffixes=('','_neg'))
                .query('cnt_neg.isnull()')
                [['tag', 'cnt']]
            )
        if user_positive_tags.shape[0] > 0:
            tags_df = (
                tags_df
                .merge(user_positive_tags, how='left', left_on='tag', right_on='content_tag',suffixes=('','_pos'))
                .sort_values(['cnt_pos', 'cnt'], ascending=[False, False])
                [['tag', 'cnt', 'cnt_pos']]
            ).head(5)  # add
    return tags_df

user_tags = user_tags_ranking(user_history, tags_df)

user_tags

,tag,cnt,cnt_pos
10,minimalism,143,1.0
51,art neo-pop,29,1.0
65,transavantgarde,17,1.0
78,precisionism,14,1.0
0,expressionism,558,NaN


cnt_pos - счетчик позитивов. Дополнительно поднимаем наверх наиболее популярные тэги среди релевантных

Если нет релевантных тэгов - показываем просто популярные

Реализуем механику рекомендаций - по наилучшему тэгу подбираем подходящий контент

Чтобы увеличить recall будем искать не по одному тэгу, а по топу лучших тэгов

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer='word',
    lowercase=True,
    token_pattern=r'\b[\w\d]{3,}\b'
)

vectorizer.fit(content_db_df['art_tags'].values)
corpus_numpy = vectorizer.transform(content_db_df['art_tags'].values)

In [82]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

def recommend(query, corpus_numpy, num_neighbors=5):
    query_embed = vectorizer.transform([query])
    similarities = cosine_similarity(query_embed, corpus_numpy).flatten()
    closest_indexes = np.argsort(similarities)[::-1][:num_neighbors]

    return closest_indexes

query = ' '.join(user_tags['tag'].values)
rec_ids = recommend(query, corpus_numpy)

content_db_df.iloc[rec_ids][cols]

,artist_name,art_tags,art movement,field
760,Enzo Cucchi,"neo-expressionism,transavantgarde,neo-expressi...","transavantgarde neo-expressionism , neo-expres...",painting
3245,Valeria Trubina,"transavantgarde,neo-expressionism,transavantgarde","transavantgarde, neo-expressionism neo-express...",painting
1465,Oleg Holosiy,"transavantgarde,neo-expressionism,transavantgarde","transavantgarde, neo-expressionism neo-express...",painting
2728,Vasiliy Ryabchenko,"transavantgarde,expressionism,contemporary,neo...","expressionism contemporary transavantgarde , n...","drawing painting, drawing, installation, colla..."
1793,Martyl Langsdorf,"precisionism,expressionism,expressionism,preci...","precisionism , abstract expressionism, precisi...",painting


# System design

Дизайн системы - процесс проектирования ML сервиса

По сути мы декомпозируем наш алгоритм, описанный в разделе business logic и реализуем отдельные компоненты системы с помощью подходящих технологий

* Client - Пользовательский интерфейс системы. Для реализации используем Streamlit
* ML Bandit API - "сердце" сервиса, механизм рекомендаций. Для реализации используем FastAPI
* UserActionHistory - база данных, где храним историю пользователя. Для реализации используем Mongo.
* ContentCatalog - каталог, хранилище контента который показываем пользователю. Для реализации используем Pandas.